# Documentation  

Anaconda 2019.07 버전의 python 3.7.4 defalut 값으로 진행  
  
Python = 3.7.4  
Pandas = 0.25.1   
Numpy = 1.16.5  
urllib3 = 1.24.2  
BeautifulSoup = 4.8.0  
requests = 2.22.0   
selenium = 3.141.0  

In [1]:
#import module

#python defalut module
import time
import re
import os
from datetime import datetime,timedelta
from IPython.display import Image

import pandas as pd
import numpy as np

import urllib3
from bs4 import BeautifulSoup
import requests

import selenium

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

import warnings
warnings.filterwarnings(action='ignore')

In [1]:
# 이전 문서와 중복되면 해당 기사의 row index를 기록하는 코드
def duplicate_title(prev_list,prev_title,save_list) :
    for row, title in enumerate(prev_title) :
        if title in prev_list :
            save_list.append(row) 


    
def article_post(news_title,news_text) :
    
    icu_title = driver.find_element_by_id("Title_fa564e0f-0c70-4ab9-b863-0177e6ddd247_$TextField") # 제목란 클릭

    icu_title.send_keys(news_title) # 작성 제목에 기사 제목 입력
    time.sleep(5)
    
    # iframe 전환

    # 작성 게시글 내부의 inline frame(iframe) 2개의 web page가 삽입되어 있습니다.
    # 기사를 입력하기 위해서는 내부에 들어가서 작성해야 합니다.
    # 이를 위해 tag_name으로 iframe이 입력된 전체 리스트를 찾고 
    # 기사 제목을 입력할 수 있는 iframe[0] 내부의 두번째 iframe에 접속하여 기사 본문을 게시하는 코드를 작성하였습니다.
    
    iframes = driver.find_elements_by_tag_name('iframe') # 모든 iframe tage_name을 검색
    driver.switch_to.frame(iframes[0]) # 그 중 본문을 작성할 수 있는 첫번째 iframe 선택
    time.sleep(2)

    second_iframe = driver.find_elements_by_tag_name('iframe') # 본문에 접속하기 위해서는 iframe[0]내부의 iframe에 접속하여 작성해야됨
    driver.find_element_by_xpath('//*[@id="ue_dext5editorsource"]/a').click() # I.C.U 본문작성 하위 태그의 HTML태그 클릭 
    # 이를 위해 2번째 iframe을 검색
    driver.switch_to.frame(second_iframe[0])  # 검색한 두 번째 iframe 접속
    icu_news = driver.find_element_by_css_selector("#dext_body") # 해당 iframe 내의 기사를 입력할 수 있는 id = dext_body 검색
    icu_news.clear()
    icu_news.send_keys(news_text) # 해당 부분에 기사 내용 입력
    time.sleep(5)
    
    driver.switch_to_default_content()# 해당 iframe에 벗어나 상위 웹페이지로 연결


In [10]:
final_df = pd.read_csv('C:/driver/news_save/finaldf.csv',encoding='UTF-8')
final_title = final_df.title.tolist()

### Selenium활용하여 Autoway에 자동 로그인 후 커뮤니티 게시
> IT기사 중 관련 없는 기사, 과학 기사 등 다양한 변수가 존재하여, 내부 검수가 필요하다고 판단하였습니다.  
>  
> 이를 위해 커뮤니티에 기사를 게시 후, 내부 검수하여 관련 없는 기사를 삭제하고, 
>  
> 일정 시간 이후에 남은 기사를 I.C.U에 게시하는 방향으로 정했습니다

In [4]:
set_id = open("C:/driver/set_id.txt", 'r').read() #id 입력
set_pwd = open("C:/driver/pwd.txt", 'r').read() # pwd 입력

In [6]:
def resource_path(relative_path):
    try: 
        base_path = sys._MEIPASS
    except Exception:
        base_path = os.path.abspath(".")
    return os.path.join(base_path, relative_path)

chromedriver_path = resource_path("chromedriver.exe")


### 커뮤니티에서 내부한 검수 뉴스를 I.C.U에 게시하는 코드

In [49]:
# Selenium Load
driver = webdriver.Chrome(chromedriver_path)
driver.maximize_window() # chrome driver 전체 화면 설정

# site login
driver.get('https://autoway.hyundai.net/') # 오토웨이 접속
time.sleep(3)

driver.find_element_by_xpath('//*[@id="form1"]/div[3]/div[2]/div[3]/div/div[3]').click() 
# autoway접속하여 로그인창의 부품 선택

driver.find_element_by_xpath('//*[@id="drpGroupCopList"]/option[1]').click() 
# 부품의 현대트랜시스 선택


log = driver.find_element_by_id("userID") # 오토웨이 id 입력 주소
log.send_keys(set_id) # id입력
time.sleep(2)
pwd = driver.find_element_by_id("password") # 오토웨이 비밀번호 입력 주소
pwd.send_keys(set_pwd) # pwd 입력
time.sleep(2)
driver.find_element_by_xpath('//*[@id="Login"]').click() # 로그인 버튼 클릭
time.sleep(5)

driver.get('https://autoway.hyundai.net/comm/C00621/Lists/B00002/AllItems.aspx')
time.sleep(6)

res = driver.page_source
community_html = BeautifulSoup(res,'html.parser') # html 태그 기준으로 parsing
content = community_html.select(".c-table__ellipsis") # 기사 본문 저장되어 있는 html주소 복사

inspect_list = []

# 추출한 태그 내부의 시스템 운영 ICT기획팀 등 다른 tag가 같이 저장됨 해당 tag가 나올 경우 skip

for i in content:
    if (i.string == '시스템운영') | (i.string == 'ICT기획팀') :
        continue
    com_str = i.string.replace('\xa0','')
    inspect_list.append(com_str)
    

com_row = []
duplicate_title(inspect_list,final_title,com_row)

final_df = final_df.loc[com_row,:]
final_title = final_df.title.tolist()
article_list = final_df.article.tolist()

time.sleep(1)
driver.get('https://autoway.hyundai.net/board/H104/Lists/H10400357/TileListView.aspx') # 현대트랜시스 I.C.U주소 입력하여 접속
time.sleep(6)

for final_icu_title, final_icu_article in zip(final_df.title, final_df.article) :

    driver.find_element_by_xpath('//*[@id="ctl00_ctl59_g_de894c14_b56d_4d4b_9c89_374a6c0455c1_Search_NewButton_New_ctl00_ctl59_g_de894c14_b56d_4d4b_9c89_374a6c0455c1_Search_NewButton"]/span').click()
    # I.C.U 내부의 작성 버튼 클릭
    time.sleep(7)

    article_post(final_icu_title,final_icu_article)
    time.sleep(1)
    # 기사 최종 게시 클릭

    driver.find_element_by_xpath('//*[@id="ctl00_ctl59_g_0005fe00_cee6_4765_9070_f77f554d027e_ctl00_ctl06_btnSave"]').click()
    

    
    # 임시저장 코드
#     driver.find_element_by_xpath('//*[@id="ctl00_ctl59_g_0005fe00_cee6_4765_9070_f77f554d027e_ctl00_ctl05_btnTempSave"]').click()
    time.sleep(3)
    


In [ ]:
driver.quit()